In [1]:
%%writefile EncryptSHA512.c
#include <string.h>
#include <stdio.h>
#include <stdlib.h>
#include <crypt.h>
#include <unistd.h>

#define SALT "$6$AS$"

int main(int argc, char *argv[]){

  printf("%s\n", crypt(argv[1], SALT));

  return 0;
}

Writing EncryptSHA512.c


In [ ]:
!gcc -o EncryptSHA512 EncryptSHA512.c -lcrypt

In [ ]:
!./EncryptSHA512 AA10

$6$AS$KDz2YK3J483TubP5vGxy79Xa1cmV8e75Asy0ekjoV7HtuS7EAjnH.xWmVng0IhoIg12JEzlnHFcu9UzS1tbXk1


In [ ]:
%%writefile PasswordCracking.c
#include <stdio.h>
#include <stdlib.h>
#include <crypt.h>
#include <pthread.h>
#include <string.h>
#include <stdatomic.h>

#define SALT "$6$AS$"
#define THREAD_COUNT 4

atomic_int found = 0;

struct ThreadArgs {
    int thread_id;
    const char *encryptedPassword;
    char foundPassword[5];
};

char* encryptPassword(char* password, struct crypt_data* cdata) {
    return crypt_r(password, SALT, cdata);
}

void* crackPassword(void* arguments) {
    struct ThreadArgs* args = (struct ThreadArgs*)arguments;
    char password[5] = "AA00", *encryptedGuess;
    struct crypt_data cdata = {0};

    for (int i = args->thread_id; i < 26 * 26 * 100 && !found; i += THREAD_COUNT) {
        int num = i;
        password[3] = '0' + (num % 10); num /= 10;
        password[2] = '0' + (num % 10); num /= 10;
        password[1] = 'A' + (num % 26); num /= 26;
        password[0] = 'A' + (num % 26);

        printf("Thread %d: Trying: %s\n", args->thread_id, password);

        encryptedGuess = encryptPassword(password, &cdata);
        if (strcmp(args->encryptedPassword, encryptedGuess) == 0) {
            strncpy(args->foundPassword, password, 5);
            atomic_store(&found, 1);
            printf("Password found: %s\n", args->foundPassword);
            return NULL;
        }
    }
    return NULL;
}

int main() {
    char inputPassword[100];
    printf("Enter the encrypted password: ");
    scanf("%s", inputPassword);

    pthread_t threads[THREAD_COUNT];
    struct ThreadArgs threadArgs[THREAD_COUNT];

    for (int i = 0; i < THREAD_COUNT; i++) {
        threadArgs[i].thread_id = i;
        threadArgs[i].encryptedPassword = inputPassword;
        if (pthread_create(&threads[i], NULL, crackPassword, (void*)&threadArgs[i]) != 0) {
            fprintf(stderr, "Failed to create thread.\n");
            return EXIT_FAILURE;
        }
    }

    for (int i = 0; i < THREAD_COUNT; i++) {
        if (pthread_join(threads[i], NULL) != 0) {
            fprintf(stderr, "Failed to join thread.\n");
            return EXIT_FAILURE;
        }
    }

    if (!found) {
        printf("Password not found.\n");
    }

    return 0;
}

Writing PasswordCracking.c


In [ ]:
!gcc -o passwordCracking PasswordCracking.c -lcrypt -lpthread

In [ ]:
!./passwordCracking

Enter the encrypted password: $6$AS$KDz2YK3J483TubP5vGxy79Xa1cmV8e75Asy0ekjoV7HtuS7EAjnH.xWmVng0IhoIg12JEzlnHFcu9UzS1tbXk1
Thread 1: Trying: AA01
Thread 0: Trying: AA00
Thread 2: Trying: AA02
Thread 0: Trying: AA04
Thread 3: Trying: AA03
Thread 1: Trying: AA05
Thread 2: Trying: AA06
Thread 3: Trying: AA07
Thread 0: Trying: AA08
Thread 2: Trying: AA10
Thread 1: Trying: AA09
Thread 3: Trying: AA11
Password found: AA10
